# Pytorch를 활용한 Resnet50 트랜스퍼 러닝
## 라이브러리 임포트


In [2]:
import torch
from torchvision import utils
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [3]:
import os

os.makedirs('./train', exist_ok=True)
os.makedirs('./test', exist_ok=True)

train_dataset = datasets.STL10('./train', split='train', download=True,
                              transform=transforms.ToTensor())
test_dataset = datasets.STL10('./test', split='test', download=True,
                              transform=transforms.ToTensor())

  0%|          | 0/2640397119 [00:00<?, ?it/s]

Extracting ./train/stl10_binary.tar.gz to ./train


  0%|          | 0/2640397119 [00:00<?, ?it/s]

Extracting ./test/stl10_binary.tar.gz to ./test


## 데이터 평균과 분산 계산

In [4]:
import numpy as np

def get_mean(dataset):
    meanRGB = [np.mean(image.numpy(), axis=(1,2)) for image, _ in dataset]
    meanR = np.mean([m[0] for m in meanRGB])
    meanG = np.mean([m[1] for m in meanRGB])
    meanB = np.mean([m[2] for m in meanRGB])
    return [meanR, meanG, meanB]

def get_std(dataset):
    stdRGB = [np.std(image.numpy(), axis=(1,2)) for image, _ in dataset]
    stdR = np.mean([m[0] for m in stdRGB])
    stdG = np.mean([m[1] for m in stdRGB])
    stdB = np.mean([m[2] for m in stdRGB])
    return [stdR, stdG, stdB]

In [5]:
train_transform = transforms.Compose([transforms.Resize((128,128)),
                                      transforms.ToTensor(),
                                      transforms.Normalize(get_mean(train_dataset),
                                                           get_std(train_dataset))])

test_transform = transforms.Compose([transforms.Resize((128,128)),
                                      transforms.ToTensor(),
                                      transforms.Normalize(get_mean(test_dataset),
                                                           get_std(test_dataset))])

train_dataset.transform = train_transform
test_dataset.transform = test_transform

In [15]:
batch_size = 64
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## 레즈넷50을 불러와서 레이어 모양을 출력

In [16]:
from torchvision import models

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # cuda가 가능하면 cuda로 
model = models.resnet50(pretrained=True).to(device)

In [17]:
!pip install torchsummary
from torchsummary import summary
summary(model, (3, 128, 128))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 64, 64]           9,408
       BatchNorm2d-2           [-1, 64, 64, 64]             128
              ReLU-3           [-1, 64, 64, 64]               0
         MaxPool2d-4           [-1, 64, 32, 32]               0
            Conv2d-5           [-1, 64, 32, 32]           4,096
       BatchNorm2d-6           [-1, 64, 32, 32]             128
              ReLU-7           [-1, 64, 32, 32]               0
            Conv2d-8           [-1, 64, 32, 32]          36,864
       BatchNorm2d-9           [-1, 64, 32, 32]             128
             ReLU-10           [-1, 64, 32, 32]               0
           Conv2d-11          [-1, 256, 32, 32]          16,384
      BatchNorm2d-12          [-1, 256, 32, 32]             512
           Conv2d-13          [-1, 256, 32, 32]          16,384
      BatchNorm2d-14          [-1, 256,

## 하이퍼 파라미터

In [18]:
import torch
import torch.nn as nn
from torch import optim

lr = 0.0001
num_epochs = 5
optimizer = optim.Adam(model.parameters(), lr=lr)
loss_function = nn.CrossEntropyLoss().to(device)

params = {
    'num_epochs': num_epochs,
    'optimizer': optimizer,
    'loss_function': loss_function,
    'train_dataloader': train_dataloader,
    'test_dataloader': test_dataloader,
    'device': device
}

## 트레인 함수 정의하기

In [19]:
def train(model, params):
    loss_function = params['loss_function']
    train_dataloader = params['train_dataloader']
    test_dataloader = params['test_dataloader']
    device = params['device']
    
    for epoch in range(num_epochs):
        # 훈련 
        for i, data in enumerate(train_dataloader, 0):
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad() # 기울기 초기화
            
            outputs = model(inputs)
            train_loss = loss_function(outputs, labels) # 손실 연산
            train_loss.backward() # 역전파
            optimizer.step() # 업데이트
    
        # 훈련 상태를 종합하여 출력 (중간 평가)
        total = 0
        correct = 0
        accuracy = []
        
        for i, data in enumerate(test_dataloader, 0):
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)            
            
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            test_loss = loss_function(outputs, labels).item()
            accuracy.append(100 * correct/total)
        
        print(f'''Epoch: {epoch+1}/{num_epochs}, Train loss: {train_loss.item():.6f}, \
Test loss: {test_loss:.6f}, Accuracy: {100*correct/total:.2f}''')
        
        
train(model, params)

Epoch: 1/5, Train loss: 0.440862, Test loss: 0.252152', Accuracy: 90.17
Epoch: 2/5, Train loss: 0.619012, Test loss: 0.366811', Accuracy: 91.19
Epoch: 3/5, Train loss: 0.033354, Test loss: 0.246264', Accuracy: 91.49
Epoch: 4/5, Train loss: 0.062189, Test loss: 0.139375', Accuracy: 91.75
Epoch: 5/5, Train loss: 0.212947, Test loss: 0.202763', Accuracy: 91.86
